## # To do 부정리뷰 워드클라우드

부정리뷰 워드클라우드를 생성해주세요!

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk
# nltk.download('all') # 처음 실행 시 주석을 제거하고 실행해주세요!

In [2]:
data = pd.read_csv('data/train.csv') # train 데이터 불러오기
data.head() # 데이터 확인하기

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [3]:
data.isna().sum() # 결측치 확인

id          0
document    0
label       0
dtype: int64

In [4]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(data)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
val.reset_index(inplace=True)

In [5]:
train # train 셋 확인

,index,id,document,label
0,709,710,탐 성님 폭스 형님과 100분 토론 잘봤습니다.,1
1,2621,2622,ㅋㅋㅋ여자분 몸매가 아주 쩜ㅋㅋ 액션연기도 잘하고 추천~!!,1
2,198,199,"재미도 없고 영국인, 역사 속 인물들을 이용한 파렴치한 농담 따먹기.",0
3,65,66,어린시절.아직도 잊혀지지 않는 아름다운 사랑이야기....,1
4,4102,4103,재밌게봐서 7점정도 주고싶은데 물타기에 저도 4점줍니다,0
...,...,...,...,...
3745,1811,1812,재미읍다..말장난만 겁나 하고..,0
3746,4932,4933,에로영화라도 이리재미없는 영화는 난반대일세,0
3747,1492,1493,무늬만 B급 영화인 쓰레기. 차라리 왕룡감독의 북두의권 실사를 봐라.,0
3748,1337,1338,프롯은 케이팩스로 돌아갔을것이다. 은근히 여운이 남는 영화.,1


In [6]:
train['preprocessed'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
train[:5] # 데이터 확인

,index,id,document,label,preprocessed
0,709,710,탐 성님 폭스 형님과 100분 토론 잘봤습니다.,1,탐 성님 폭스 형님과 분 토론 잘봤습니다
1,2621,2622,ㅋㅋㅋ여자분 몸매가 아주 쩜ㅋㅋ 액션연기도 잘하고 추천~!!,1,ㅋㅋㅋ여자분 몸매가 아주 쩜ㅋㅋ 액션연기도 잘하고 추천
2,198,199,"재미도 없고 영국인, 역사 속 인물들을 이용한 파렴치한 농담 따먹기.",0,재미도 없고 영국인 역사 속 인물들을 이용한 파렴치한 농담 따먹기
3,65,66,어린시절.아직도 잊혀지지 않는 아름다운 사랑이야기....,1,어린시절아직도 잊혀지지 않는 아름다운 사랑이야기
4,4102,4103,재밌게봐서 7점정도 주고싶은데 물타기에 저도 4점줍니다,0,재밌게봐서 점정도 주고싶은데 물타기에 저도 점줍니다


In [7]:
train['preprocessed'] = train['preprocessed'].str.replace("[\s]+", " ") # 다중 공백 제거
train[:5] # 데이터 확인

,index,id,document,label,preprocessed
0,709,710,탐 성님 폭스 형님과 100분 토론 잘봤습니다.,1,탐 성님 폭스 형님과 분 토론 잘봤습니다
1,2621,2622,ㅋㅋㅋ여자분 몸매가 아주 쩜ㅋㅋ 액션연기도 잘하고 추천~!!,1,ㅋㅋㅋ여자분 몸매가 아주 쩜ㅋㅋ 액션연기도 잘하고 추천
2,198,199,"재미도 없고 영국인, 역사 속 인물들을 이용한 파렴치한 농담 따먹기.",0,재미도 없고 영국인 역사 속 인물들을 이용한 파렴치한 농담 따먹기
3,65,66,어린시절.아직도 잊혀지지 않는 아름다운 사랑이야기....,1,어린시절아직도 잊혀지지 않는 아름다운 사랑이야기
4,4102,4103,재밌게봐서 7점정도 주고싶은데 물타기에 저도 4점줍니다,0,재밌게봐서 점정도 주고싶은데 물타기에 저도 점줍니다


In [8]:
from konlpy.tag import Okt
okt = Okt()

In [9]:
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in train['preprocessed']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
    # To do
    tokens = okt.morphs(sentence, stem=True)# 형태소 분석 (어간 추출로 옵션 설정)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
train["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,index,id,document,label,preprocessed,tokenized_stem
0,709,710,탐 성님 폭스 형님과 100분 토론 잘봤습니다.,1,탐 성님 폭스 형님과 분 토론 잘봤습니다,타다 성 님 폭스 형님 과 분 토론 잘 보다
1,2621,2622,ㅋㅋㅋ여자분 몸매가 아주 쩜ㅋㅋ 액션연기도 잘하고 추천~!!,1,ㅋㅋㅋ여자분 몸매가 아주 쩜ㅋㅋ 액션연기도 잘하고 추천,ㅋㅋㅋ 여자 분 몸매 가 아주 쩜 ㅋㅋ 액션연기 도 잘 하다 추천
2,198,199,"재미도 없고 영국인, 역사 속 인물들을 이용한 파렴치한 농담 따먹기.",0,재미도 없고 영국인 역사 속 인물들을 이용한 파렴치한 농담 따먹기,재미 도 없다 영국인 역사 속 인물 들 을 이용 한 파렴치하다 농담 따먹다
3,65,66,어린시절.아직도 잊혀지지 않는 아름다운 사랑이야기....,1,어린시절아직도 잊혀지지 않는 아름다운 사랑이야기,어린시절 아직도 잊혀지다 않다 아름답다 사랑 이야기
4,4102,4103,재밌게봐서 7점정도 주고싶은데 물타기에 저도 4점줍니다,0,재밌게봐서 점정도 주고싶은데 물타기에 저도 점줍니다,재밌다 보다 점 정도 주다 물 타다 저 도 점 줍다


In [10]:
#### 품사 사용 범위 확인

main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in train['document']: # 리뷰들을 하나씩 가져옵니다
    # To do
    pos = okt.pos(sentence)# 품사 태깅으로 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다
train["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,709,710,탐 성님 폭스 형님과 100분 토론 잘봤습니다.,1,탐 성님 폭스 형님과 분 토론 잘봤습니다,타다 성 님 폭스 형님 과 분 토론 잘 보다,탐 성 폭스 형님 토론 봤습니다
1,2621,2622,ㅋㅋㅋ여자분 몸매가 아주 쩜ㅋㅋ 액션연기도 잘하고 추천~!!,1,ㅋㅋㅋ여자분 몸매가 아주 쩜ㅋㅋ 액션연기도 잘하고 추천,ㅋㅋㅋ 여자 분 몸매 가 아주 쩜 ㅋㅋ 액션연기 도 잘 하다 추천,여자 몸매 아주 쩜 액션연기 하고 추천
2,198,199,"재미도 없고 영국인, 역사 속 인물들을 이용한 파렴치한 농담 따먹기.",0,재미도 없고 영국인 역사 속 인물들을 이용한 파렴치한 농담 따먹기,재미 도 없다 영국인 역사 속 인물 들 을 이용 한 파렴치하다 농담 따먹다,재미 없고 영국인 역사 속 인물 이용 파렴치한 농담 따먹기
3,65,66,어린시절.아직도 잊혀지지 않는 아름다운 사랑이야기....,1,어린시절아직도 잊혀지지 않는 아름다운 사랑이야기,어린시절 아직도 잊혀지다 않다 아름답다 사랑 이야기,어린시절 아직도 잊혀지지 않는 아름다운 사랑 이야기
4,4102,4103,재밌게봐서 7점정도 주고싶은데 물타기에 저도 4점줍니다,0,재밌게봐서 점정도 주고싶은데 물타기에 저도 점줍니다,재밌다 보다 점 정도 주다 물 타다 저 도 점 줍다,재밌게 봐서 점 정도 주고싶은데 물 타기에 저 점 줍니다


In [11]:
X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.label #training 데이터에서 라벨 추출

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
# To do
vectorizer = CountVectorizer() #countvectorizer 생성
# countvectorizer 학습
vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train) # transform

In [13]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression() #객체에 모델 할당
model.fit(X_train_vec, y_train) #모델 학습

LogisticRegression()

### 검증셋으로 모델 성능 검증

먼저 검증 셋 val에 train 셋과 동일한 전처리 과정을 거쳐줍니다.

In [14]:
# preprocessed
val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
val['preprocessed'] = val['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in val['preprocessed']:
    # To do
    tokens = okt.morphs(sentence, stem=True) #형태소 분석 어간 추출 옵션
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
val["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in val['document']:
    # To do
    pos = okt.pos(sentence) # 품사 태깅 형태소 분석
    main_words = [word[0] for word in pos if word[1] in ('Noun', 'Adjective', 'Adverb', 'Verb')]
     # 원하는 품사 선택
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)

val.head()

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,3023,3024,진짜로 돈아까운 영화라는건 부정할수 없는 사실.,0,진짜로 돈아까운 영화라는건 부정할수 없는 사실,진짜 로 돈 아깝다 영화 라는 건 부정 하다 없다 사실,진짜 돈 아까운 영화 건 부정 할수 없는 사실
1,4060,4061,영화자체가 아름답다라는 문장하나로 표현이 되네요!,1,영화자체가 아름답다라는 문장하나로 표현이 되네요,영화 자체 가 아름답다 문장 하나로 표현 이 되다,영화 자체 아름답다라는 문장 하나로 표현 되네요
2,683,684,너무 /뻔하고 잔잔해요 생각보다 편집이 별로네요,0,너무 뻔하고 잔잔해요 생각보다 편집이 별로네요,너무 뻔하다 잔잔하다 생각 보다 편집 이 별로 네 요,너무 뻔하고 잔잔해요 생각 편집 별로
3,825,826,최고다 //스포츠 영화에 빠져들게 한 영화,1,최고다 스포츠 영화에 빠져들게 한 영화,최고다 스포츠 영화 에 빠져들다 하다 영화,최고다 스포츠 영화 빠져들게 한 영화
4,4502,4503,케이블에서 방송되서아쉬웠...지상파에서했으면 시청률팍팍나왔을텐데...,1,케이블에서 방송되서아쉬웠지상파에서했으면 시청률팍팍나왔을텐데,케이블 에서 방송 되다 아쉽다 지상파 에서 하다 시청률 팍팍 나오다,케이블 방송 되서 아쉬웠 지상파 했으면 시청률 팍팍 나왔을텐데


학습한 모델로 예측할 X를 벡터화해줍니다.

In [15]:
X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.label #validation 데이터에서 라벨 추출
# To do
X_val_vec = vectorizer.transform(X_val)# train셋으로 fit한 벡터라이저 이용해 transform

In [16]:
#run model
y_pred = model.predict(X_val_vec)
print(y_pred)
#0:negative 1:positive

[0 1 0 ... 0 0 1]


In [17]:
from sklearn import metrics
print('accuracy =', metrics.accuracy_score(y_val, y_pred)) # 정확도 확인

accuracy = 0.82


# test.csv 분류하기

In [18]:
test = pd.read_csv('data/test.csv')
test

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다
...,...,...
4995,4996,연계도 제대로 안되고 뭔가 부족한느낌이들고 찝찝한영화
4996,4997,내생에 최악의 영화중 하나 졸려서 미칠뻔....
4997,4998,지금까지 나온 드라마 중 최고였습니다. 100점 만점!
4998,4999,영화값으로 엄청난 공연을 보았다 - 이게 웬 횡재 ..


동일한 전처리 과정을 거치되, data leakage가 발생하지 않도록 주의합니다.

In [19]:
test.isna().sum()

id          0
document    0
dtype: int64

In [ ]:
# preprocessed
test['preprocessed'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
test['preprocessed'] = test['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in test['preprocessed']:
    # To do
    tokens = okt.morphs(sentence, stem=True)#형태소 분석, 어간 추출
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
test["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in test['document']:
  # To do
    pos = okt.pos(sentence)#품사 태그
    main_words = [word[0] for word in pos if word[1] in ['Noun', 'Adverb', 'Adjective', 'Verb']] # 원하는 품사 추출
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
test["main_pos"] = pd.DataFrame(main_pos)

test.head()

In [ ]:
X_test = test.main_pos
# To do
X_test_vec = vectorizer.transform(X_test) # train셋으로 fit한 벡터라이저 이용해 transform
pred_test = model.predict(X_test_vec)
print(pred_test)

# 워드클라우드

In [ ]:
test['pred']=pred_test

In [ ]:
test

In [ ]:
df=test.copy()

In [ ]:
import pandas as pd
import re

from wordcloud import WordCloud, STOPWORDS

from matplotlib import font_manager, rc
import matplotlib.pyplot as plt

from konlpy.tag import Okt
from collections import Counter

import matplotlib.font_manager as fm

fontpath = 'data/BMHANNA.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='BMHANNA')

In [ ]:
pos = df[df['pred']==1] #긍정리뷰
neg = df[df['pred']==0] #부정리뷰

In [ ]:
display(pos.head())
display(neg.head())

## 부정리뷰 워드클라우드

In [ ]:
okt = Okt()

content_list = list(neg['main_pos'])
noun_list = []

for sentence in content_list:


    sentence = re.sub('[a-zA-z0-9]','', sentence)
    
    for word, tag in okt.pos(sentence):
      # 명사만 추출
        if tag in ['Noun']:  

            if len(word) == 1:
                continue

            else:
                noun_list.append(word)

count = Counter(noun_list)
negwords = dict(count.most_common(n=50)) 

In [ ]:
negwords

In [ ]:
negwords.items()

In [ ]:
#특정 단어 지정해서 불용어 제거
for w in ['진짜', '영화', '이건', '정도']:
    negwords.pop(w)

In [ ]:
# 불용어 리스트 생성해서 제거
stop_words = "다시 완전 마지막 지금 역시 영화"
stop_words = stop_words.split(' ')


# 불용어 제거 실시
negword2 = [(i,j)for i,j in negwords.items()  if i not in stop_words]

# 결과 출력
negword3=dict(negword2)
print(negword3)

In [ ]:
from PIL import Image, ImageFilter

In [ ]:
Image.open("data/mask.png")

In [ ]:
import numpy as np
mask_array = np.array(Image.open("data/mask.png"))
mask_array

옵션정리

alicewc = WordCloud(
    
    background_color = "white",# 배경색 설정

    collocations = False, # 이때 collocations=True 이면 said King, said Hatter 같이 자주 나타나는 단어는 하나의 어구로 분류됨.

    stopwords = stopwords, #불용어 지정

    width = 600, height = 400, #이미지 크기 지정

    colormap = "Reds", # 글씨 색깔 지정

    #font_path = 'path입력'

    min_font_size = 5, max_font_size = 100 #폰트 지정 작게할수록 더 많은 글씨가 보이고 크게하면 글자가 강조됨. 
    
    )

In [ ]:
mask1 = np.array(Image.open("data/mask.png"))

wordcloud = WordCloud(font_path = fontpath,
    width = 800,
    height = 800,
    background_color="white",
    mask = mask1,
    min_font_size = 3
)

wordcloud = wordcloud.generate_from_frequencies(negword3)
plt.figure(figsize=(20, 20))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()